In [1]:
# import必要套件
from urllib import parse
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from time import time, sleep
from random import randint
import os
import re

C:\Program Files\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
# 傳入URL、設定隨機UserAgent、發出請求並確認狀況
url = 'https://ncode.syosetu.com/n9722do/'

# 設定隨機UserAgent
ua = UserAgent()
my_headers = {
    'User-Agent':ua.random
}

# 發出get request，確認連線狀況
response = req.get(url, headers = my_headers)
print(f'網站狀態碼: {response.status_code}')
print(f'網站編碼　: {response.encoding}')
print(f'回覆標頭　: {response.headers}')

網站狀態碼: 200
網站編碼　: UTF-8
回覆標頭　: {'Date': 'Tue, 28 Jun 2022 14:30:28 GMT', 'Server': 'Apache', 'Set-Cookie': 'ks2=zd5qm9x5xn1; expires=Wed, 28-Jun-2023 14:30:28 GMT; Max-Age=31536000; path=/; domain=syosetu.com; secure', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Connection': 'close', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=UTF-8'}


In [3]:
# 開始爬蟲 - 建立soup物件、取得小說名

# 建立soup物件用lxml parser
soup = bs(response.text, 'lxml')

# 小說標題為只有一個內容的iterable物件，用index 0取出值
novel_title = soup.select('div#novel_contents p.novel_title')[0].get_text()
print(novel_title)

最強剣士、最底辺騎士団で奮戦中　～オークを地の果てまで追い詰めて絶対に始末するだけの簡単？なお仕事です～


C:\Program Files\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [ ]:
novel_content = soup.select('div.index_box')

for content in novel_content:
    print(content)

In [ ]:
# 確認小說本體(div.index_box內部構造)
novel_content = soup.select('div.index_box')

regexChapterTitle = r'<div class=\"chapter_title\">([一-龯ぁ-んァ-ン　\w]+)<\/div>'
# index_box內為一個巨大的content串列 (只有一個元素)
# for element in novel_content:
#     print(f"=====\n{element}\n=====")

# print(novel_content)
print(re.search(regexChapterTitle, str(novel_content))[1])

print(f"novel_content元素類型: {type(novel_content)}")
print(f"novel_content內元素數: {len(novel_content)}")
print(f"novel_content內文: \n{novel_content}")

In [8]:
# div.index_box內第一個層級有div以及dl 2種元素標籤，設定讓select 2種都算
novel_content = soup.select('div.index_box')
content = novel_content[0].select('div, dl')

for item in content[0:3]:
    print(f"item: {item}")
    print("="*50)

print("\n\n===== 分隔線 =====\n\n")

novel_content2 = soup.select('div.index_box div,dl')
for item in novel_content2[0:3]:
    print(f"item: {item}")
    print("="*50)

item: <div class="chapter_title">第一章　そろそろ入団3年目</div>
item: <dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n9722do/1/">1.物語の始まりです</a>
</dd>
<dt class="long_update">
2016/10/16 22:00<span title="2020/11/23 01:30 改稿">（<u>改</u>）</span></dt>
</dl>
item: <dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n9722do/2/">2.殺せばいい訳ではありません</a>
</dd>
<dt class="long_update">
2016/11/25 02:42<span title="2020/01/31 15:15 改稿">（<u>改</u>）</span></dt>
</dl>


===== 分隔線 =====


item: <div class="chapter_title">第一章　そろそろ入団3年目</div>
item: <dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n9722do/1/">1.物語の始まりです</a>
</dd>
<dt class="long_update">
2016/10/16 22:00<span title="2020/11/23 01:30 改稿">（<u>改</u>）</span></dt>
</dl>
item: <dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n9722do/2/">2.殺せばいい訳ではありません</a>
</dd>
<dt class="long_update">
2016/11/25 02:42<span title="2020/01/31 15:15 改稿">（<u>改</u>）</span></dt>
</dl>


In [37]:
novel_content = soup.select('div.index_box div,dl')
chapterInfo = []
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'

# for item in novel_content:
#     print(item)
#     print("=" * 50)

section = ''
for item in novel_content[0:20]:
    if item['class']==['chapter_title']:
        section = item.get_text()
        continue
    else:
        chapterTitle = item.a.get_text()
        uploadDateTime = re.search(regexDateTime, item.dt.get_text())[0]
        updateDateTime = ''
        if item.find('span') != None:
            updateDateTime = re.search(regexDateTime, item.span['title'])[0]
        chapterLink = f"https://ncode.syosetu.com{item.a['href']}"

    link = [section, chapterTitle, uploadDateTime, updateDateTime, chapterLink]
    chapterInfo.append(link)

# 確認抓到資料(for-each)
for item in chapterInfo:
    print(f"章名:{item[0]}　節名:{item[1]}　上傳時間:{item[2]}　更新時間:{item[3]}　連結:{item[4]}")

章名:第一章　そろそろ入団3年目　節名:1.物語の始まりです　上傳時間:2016/10/16 22:00　更新時間:2020/11/23 01:30　連結:https://ncode.syosetu.com/n9722do/1/
章名:第一章　そろそろ入団3年目　節名:2.殺せばいい訳ではありません　上傳時間:2016/11/25 02:42　更新時間:2020/01/31 15:15　連結:https://ncode.syosetu.com/n9722do/2/
章名:第一章　そろそろ入団3年目　節名:3.鎧なんて飾りです　上傳時間:2016/10/18 22:00　更新時間:2020/01/31 15:15　連結:https://ncode.syosetu.com/n9722do/3/
章名:第一章　そろそろ入団3年目　節名:4.苦労の割に安月給です　上傳時間:2016/10/20 17:00　更新時間:2020/01/31 15:15　連結:https://ncode.syosetu.com/n9722do/4/
章名:第一章　そろそろ入団3年目　節名:5.魔法で敵は倒せません　上傳時間:2016/10/22 15:00　更新時間:2020/01/31 15:16　連結:https://ncode.syosetu.com/n9722do/5/
章名:第一章　そろそろ入団3年目　節名:6.責任はやたらと重大です　上傳時間:2016/10/24 15:00　更新時間:2021/04/24 20:30　連結:https://ncode.syosetu.com/n9722do/6/
章名:第二章　王都で御前試合　節名:7.人参は経費で落ちません　上傳時間:2016/10/26 15:00　更新時間:2020/01/31 15:17　連結:https://ncode.syosetu.com/n9722do/7/
章名:第二章　王都で御前試合　節名:8.絆は時として残酷です　上傳時間:2016/10/28 15:00　更新時間:2020/09/14 21:13　連結:https://ncode.syosetu.com/n9722do/8/
章名:第二章　王都で御前試合　節名:9.無駄な知識は豊富です　上傳時間:2016/10/30 16:00　更新時間:2020/01/3

In [40]:
# 抓出章名以及其中小說節名
novel_content = soup.select('div.index_box')
regexChapterTitle = r'<div class=\"chapter_title\">([一-龯ぁ-んァ-ン　\w]+)<\/div>'
chapterInfoSource = soup.select('div.index_box dl.novel_sublist2')
chapterInfo = []
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'
howMany = 20

# 不限制取出筆數
# for item in chapterInfoSource:
#     sectionSliceIndex = str(novel_content).find(item.a.get_text())
#     sectionString = str(novel_content)[0:sectionSliceIndex]
#     section = re.findall(regexChapterTitle, sectionString)
#     latest_section = section[len(section)-1]
#     chapterTitle = item.a.get_text()
#     uploadDateTime = re.search(regexDateTime, item.dt.get_text())[0]
#     updateDateTime = ''
#     if item.find('span') != None:
#         updateDateTime = re.search(regexDateTime, item.span['title'])[0]
#     chapterLink = f"https://ncode.syosetu.com{item.a['href']}"

#     link = [latest_section, chapterTitle, uploadDateTime, updateDateTime, chapterLink]
#     chapterInfo.append(link)


# 限制取出筆數
for i in range(howMany):
    sectionSliceIndex = str(novel_content).find(chapterInfoSource[i].a.get_text())
    sectionString = str(novel_content)[0:sectionSliceIndex]
    section = re.findall(regexChapterTitle, sectionString)
    latest_section = section[len(section)-1]
    chapterTitle = chapterInfoSource[i].a.get_text()
    uploadDateTime = re.search(regexDateTime, chapterInfoSource[i].dt.get_text())[0]
    updateDateTime = ''
    if chapterInfoSource[i].find('span') != None:
        updateDateTime = re.search(regexDateTime, chapterInfoSource[i].span['title'])[0]
    chapterLink = f"https://ncode.syosetu.com{chapterInfoSource[i].a['href']}"


    link = [latest_section, chapterTitle, uploadDateTime, updateDateTime, chapterLink]
    chapterInfo.append(link)

    
# 確認抓到資料        
# i = 0
# for i in range(50):
#     print(f"{i+1}. {chapterInfo[i]}")


# 確認抓到資料(for-each)
for item in chapterInfo:
    print(f"章名:{item[0]}　節名:{item[1]}　上傳時間:{item[2]}　更新時間:{item[3]}　連結:{item[4]}")



# 一筆一筆抓出每個
# for title in chapter_titles:
#     sectionSliceIndex = str(novel_content).find(title.a.get_text())
#     sectionString = str(novel_content)[0:sectionSliceIndex]
#     section = re.findall(regexChapterTitle, sectionString)
#     latest_section = section[len(section)-1]
#     print(f"{latest_section}　　{title.a.get_text()}")

章名:第一章　そろそろ入団3年目　節名:1.物語の始まりです　上傳時間:2016/10/16 22:00　更新時間:2020/11/23 01:30　連結:https://ncode.syosetu.com/n9722do/1/
章名:第一章　そろそろ入団3年目　節名:2.殺せばいい訳ではありません　上傳時間:2016/11/25 02:42　更新時間:2020/01/31 15:15　連結:https://ncode.syosetu.com/n9722do/2/
章名:第一章　そろそろ入団3年目　節名:3.鎧なんて飾りです　上傳時間:2016/10/18 22:00　更新時間:2020/01/31 15:15　連結:https://ncode.syosetu.com/n9722do/3/
章名:第一章　そろそろ入団3年目　節名:4.苦労の割に安月給です　上傳時間:2016/10/20 17:00　更新時間:2020/01/31 15:15　連結:https://ncode.syosetu.com/n9722do/4/
章名:第一章　そろそろ入団3年目　節名:5.魔法で敵は倒せません　上傳時間:2016/10/22 15:00　更新時間:2020/01/31 15:16　連結:https://ncode.syosetu.com/n9722do/5/
章名:第一章　そろそろ入団3年目　節名:6.責任はやたらと重大です　上傳時間:2016/10/24 15:00　更新時間:2021/04/24 20:30　連結:https://ncode.syosetu.com/n9722do/6/
章名:第二章　王都で御前試合　節名:7.人参は経費で落ちません　上傳時間:2016/10/26 15:00　更新時間:2020/01/31 15:17　連結:https://ncode.syosetu.com/n9722do/7/
章名:第二章　王都で御前試合　節名:8.絆は時として残酷です　上傳時間:2016/10/28 15:00　更新時間:2020/09/14 21:13　連結:https://ncode.syosetu.com/n9722do/8/
章名:第二章　王都で御前試合　節名:9.無駄な知識は豊富です　上傳時間:2016/10/30 16:00　更新時間:2020/01/3

In [11]:
# 抓出章名以及其中小說節名
novel_content = soup.select('div.index_box')
regexChapterTitle = r'<div class=\"chapter_title\">([一-龯ぁ-んァ-ン　\w]+)<\/div>'
chapterInfoSource = soup.select('div.index_box dl.novel_sublist2')
chapterInfoDict = {'latest_section':[], 'chapterTitle':[], 'uploadDateTime':[], 'updateDateTime':[], 'chapterLink':[]}
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'



# 限制取出筆數
for i in range(10):
    sectionSliceIndex = str(novel_content).find(chapterInfoSource[i].a.get_text())
    sectionString = str(novel_content)[0:sectionSliceIndex]
    section = re.findall(regexChapterTitle, sectionString)
    latest_section = section[len(section)-1]
    chapterTitle = chapterInfoSource[i].a.get_text()
    uploadDateTime = re.search(regexDateTime, chapterInfoSource[i].dt.get_text())[0]
    updateDateTime = ''
    if chapterInfoSource[i].find('span') != None:
        updateDateTime = re.search(regexDateTime, chapterInfoSource[i].span['title'])[0]
    chapterLink = f"https://ncode.syosetu.com{chapterInfoSource[i].a['href']}"

    chapterInfoDict['latest_section'].append(latest_section)
    chapterInfoDict['chapterTitle'].append(chapterTitle)
    chapterInfoDict['uploadDateTime'].append(uploadDateTime)
    chapterInfoDict['updateDateTime'].append(updateDateTime)
    chapterInfoDict['chapterLink'].append(chapterLink)
    
# print(chapterInfoDict)

for i in range(10):
    print(chapterInfoDict['latest_section'][i], end='')
    print(chapterInfoDict['chapterTitle'][i], end='')
    print(chapterInfoDict['uploadDateTime'][i], end='')
    print(chapterInfoDict['updateDateTime'][i], end='')
    print(chapterInfoDict['chapterLink'][i])
    print("="*50)


第一章　そろそろ入団3年目1.物語の始まりです2016/10/16 22:002020/11/23 01:30https://ncode.syosetu.com/n9722do/1/
第一章　そろそろ入団3年目2.殺せばいい訳ではありません2016/11/25 02:422020/01/31 15:15https://ncode.syosetu.com/n9722do/2/
第一章　そろそろ入団3年目3.鎧なんて飾りです2016/10/18 22:002020/01/31 15:15https://ncode.syosetu.com/n9722do/3/
第一章　そろそろ入団3年目4.苦労の割に安月給です2016/10/20 17:002020/01/31 15:15https://ncode.syosetu.com/n9722do/4/
第一章　そろそろ入団3年目5.魔法で敵は倒せません2016/10/22 15:002020/01/31 15:16https://ncode.syosetu.com/n9722do/5/
第一章　そろそろ入団3年目6.責任はやたらと重大です2016/10/24 15:002021/04/24 20:30https://ncode.syosetu.com/n9722do/6/
第二章　王都で御前試合7.人参は経費で落ちません2016/10/26 15:002020/01/31 15:17https://ncode.syosetu.com/n9722do/7/
第二章　王都で御前試合8.絆は時として残酷です2016/10/28 15:002020/09/14 21:13https://ncode.syosetu.com/n9722do/8/
第二章　王都で御前試合9.無駄な知識は豊富です2016/10/30 16:002020/01/31 15:18https://ncode.syosetu.com/n9722do/9/
第二章　王都で御前試合10.たまには本気で戦います2016/11/01 15:002020/01/29 23:56https://ncode.syosetu.com/n9722do/10/


In [ ]:
print(folderPath)

In [ ]:
# 建立章節資料夾
# illegal_c = 不能出現在資料夾名判定用字串

illegal_c = '[@_!#$%^&*()<>?/|}{~:]'


for item in chapterInfo:
    
    # ===== 建立章資料夾 =====
    chapterPath = ''
    for char in item[0]:
        if char in illegal_c:
            truncateIndex = item[0].find(char)
            chapterPath = f"{folderPath}/{item[0][0:truncateIndex]}"
            break
        else:
            chapterPath = f"{folderPath}/{item[0]}"

    if not os.path.exists(chapterPath):
        os.makedirs(chapterPath)
    # ===== 區塊結束 =====
    
    # ===== 建立爬文章本文需要的呼叫、soup物件，以及檔名 =====
    res_chapter = req.get(link[4], headers = my_headers)
    soup_chapter = bs(res_chapter.text, 'lxml')
    chapter_content = soup_chapter.select('div#novel_honbun p')
    chapter_title = item[1]
    
    for char in chapter_title:
        if char in illegal_c:
            chapter_title = chapter_title.replace(char, '')
    # ===== 區塊結束 =====
    
    # ===== 開啟檔案並寫入 =====
    with open(f'{chapterPath}/{chapter_title}.txt', 'w', encoding='utf-8') as novel_file:
        novel_file.write(f"タイトル：{item[1]}\n更新日時：{item[2]}\n最終更新：{item[3]}\n\n\n")
        for text in chapter_content:
            novel_file.write(f"{text.get_text()}\n")

    # 爬連結間空3~7秒避免因太過頻繁被認定是爬蟲
    sleep(randint(3, 7))
    # ===== 區塊結束 =====


In [ ]:
# 取得、印出各小說章節連結list
chapter_link = []
for link in soup.select('div.index_box dd.subtitle a'):
    chapter_link.append(f"https://ncode.syosetu.com{link['href']}")
    
for link in chapter_link:
    print(link)

In [ ]:
# 取得各小說章節連結 - 追加章節名、上傳時間、更新時間，存入串列的串列格式
# print(soup.select('div.index_box dl.novel_sublist2')[0])

# # 測試output用
# print(soup.select('div.index_box dl.novel_sublist2')[0].a.get_text())
# print(soup.select('div.index_box dl.novel_sublist2')[0].a['href'])
# print(soup.select('div.index_box dl.novel_sublist2')[0].dt.get_text())
# print(soup.select('div.index_box dl.novel_sublist2')[0].dt.span['title'])

chapterInfoSource = soup.select('div.index_box dl.novel_sublist2')
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'

chapterInfo = []
for item in chapterInfoSource:
    chapterTitle = item.a.get_text()
    uploadedTime = re.search(regexDateTime, item.dt.get_text())
    # uploadedTime = item.dt.get_text()
    updateTime = ''
    if item.find('span') != None:
        updateTime = re.search(regexDateTime, item.span['title'])[2]
        # updateTime = item.span['title']
    chapterLink = f"https://ncode.syosetu.com{item.a['href']}"
        
    link = [chapterTitle, uploadedTime, updateTime, chapterTitle]
            
    chapterInfo.append(link)

i = 0
for i in range(len(chapterInfo)):
    print(f"{i+1}. {chapterInfo}")

# # repr()確認是否含特殊符號
# print(repr(soup.select('div.index_box dl.novel_sublist2')[0].a.get_text()))

In [ ]:
# 印出小說標題、小說章節標題以及連結
# 小說標題

print(novel_title, end='\n\n')

# while迴圈加入印出數量限制作法
chapter_title = soup.select('div.index_box dd.subtitle a')
i=0
while i < 50:
    print(chapter_title[i].get_text(), end='')
    print(f" https://ncode.syosetu.com{chapter_title[i]['href']}")
    print('=' * 20)
    i+=1


# # for 迴圈印出全部章節標題+連結做法
# for chapter_title in soup.select('div.index_box dd.subtitle a'):
#     print(chapter_title.get_text(), end='')
#     print(f" https://ncode.syosetu.com{chapter_title['href']}")
#     print('=' * 20)

# # 印出小說連結一覽
i = 0
for i in range(50):
    print(f"{i+1}. {chapter_link[i]}")

In [ ]:
# 抓出各個章節的詳細資訊

# 從上到下依序為章節資訊、章節標題、首次上傳時間、更新時間、連結 (抓出第一筆測試)
chapter_info = soup.select('div.index_box dl.novel_sublist2')
chapter_title = chapter_info[0].a.get_text()
upload_date = chapter_info[0].dt.get_text()
update_date = chapter_info[0].dt.span['title']
novel_link = chapter_info[0].a['href']

# # print(chapter_info)
# print(chapter_title)
# print(upload_date)
# print(update_date)
# print(novel_link)

# 正規表達式 - 從抓到的資料中只抓出日期，另把日期以及時間個別群組化方便用index取用
regex_date = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'

# 印出章節名、上傳日、更新日(如果有)
for info in chapter_info:
    print(info.a.get_text())
    print(f"アップ日:{re.search(regex_date, info.dt.get_text())[1]}", end='　')
    if info.find('span') != None:
        print(f"改稿日:{re.search(regex_date, info.span['title'])[1]}")
    print(f" https://ncode.syosetu.com{info.a['href']}")

In [ ]:
# 抓取上傳時間、更新時間(更新時間為null時不印)
for info in chapter_info:
    print(info.dt.get_text(), end='')
    if info.find('span') != None:
        print(info.dt.span['title'])
        
# for info in soup.select('div.index_box dl.novel_sublist2'):
#     print(info.dt['long_update'].span['title'])

In [ ]:
# 抓取小說章節標題
section_title = soup.select('div.chapter_title')

for section in section_title:
    print(section.get_text())

In [ ]:
# # 建立存放小說的資料夾
# folderPath = novel_title.split('　')[0] # 小說的名字、去除空格後的多餘字元
# if not os.path.exists(folderPath):
#     os.makedirs(folderPath)

In [ ]:
# 從chapter_link從中抓出一篇小說

# 呼叫chapter_link第一個連結，傳出請求、建立soup物件
res_chapter = req.get(chapter_link[0], headers = my_headers)
soup_chapter = bs(res_chapter.text, 'lxml')

# 取出連結內章節的標題
chapter_title = soup_chapter.select('p.novel_subtitle')[0].get_text()
print(chapter_title, end='\n')

# 無法用於或不建議用於檔案名的非法字元
illegal_c = '[@_!#$%^&*()<>?/|}{~:]　'

# 若檔名包含非法字元，用replace移除
for char in chapter_title:
    if char in illegal_c:
        chapter_title = chapter_title.replace(char, '')

# 取出連結內章節的內文
chapter_content = soup_chapter.select('div#novel_honbun p')
for text in chapter_content:
    print(text.get_text())

In [ ]:
# # 把小說的文字存入檔案，加入換行字元改善可讀性
# with open(f'{folderPath}/{chapter_title}.txt', 'w', encoding='utf-8') as novel_file:
#     novel_file.write(f"{chapter_title}\n\n")
#     for text in chapter_content:
#         novel_file.write(f"{text.get_text()}\n")

In [ ]:
# 把小說標題切到其中不合法字元前一個index，作為儲存檔案資料夾的名字

illegal_c = '[@_!#$%^&*()<>?/|}{~:]　'
folderPath = ''

for char in novel_title:
    if char in illegal_c:
        truncateIndex = novel_title.find(char)
        folderPath = novel_title[0:truncateIndex]
        break
print(folderPath)

### 程式碼改善目標
1. 使可以判讀是否已有已抓到的章節，有時則跳過該章節
2. 設定能智慧截短小說標題避免觸發windows路徑長度限制 (在最短、合理的地方截斷標題)
3. 抓到小說更新時間放在小說內
4. 為各個章節建立子資料夾、加上編號

全角日文字比對用Regex：https://bas-man.dev/post/python/regex-double-byte-python3/

比對不在ASCII編碼中的字元：https://stackoverflow.com/questions/150033/regular-expression-to-match-non-ascii-characters

參考 - 找文字方法：https://www.delftstack.com/howto/python/position-of-character-in-string/

在bs物件內找有沒有子標籤：https://stackoverflow.com/questions/33238091/test-if-children-tag-exists-in-beautifulsoup